In [11]:
import os
from autoencoder import get_model
from config import load_config, print_config
from train import load_weights

results_dir = "/mnt/d/github/deep_learning_tutorials/03_pytorch/autoencoder_v4_1/results/unet_ae_bottle_default"
model_filename = "unet_ae_bottle_default_model.pth"
config_filename = "unet_ae_bottle_default_config.json"

config_path = os.path.join(results_dir, config_filename)
config = load_config(config_path)

model = get_model(model_type="unet_ae")
model_path = os.path.join(results_dir, model_filename)
model = load_weights(model, model_path=model_path)
model = model.to(config.device)

In [12]:
from mvtec import get_transforms, get_dataloaders

train_transform, test_transform = get_transforms(
    img_size=config.img_size,
    normalize=config.normalize
)
train_loader, valid_loader, test_loader = get_dataloaders(
    data_dir=config.data_dir,
    category=config.category,
    batch_size=config.batch_size,
    valid_ratio=config.valid_ratio,
    train_transform=train_transform,
    test_transform=test_transform
)
len(test_loader.dataset)

83

In [19]:
import torch

def denormalize(images, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    """Denormalize images from ImageNet normalization back to [0, 1] range"""
    device = images.device
    mean = torch.tensor(mean).view(1, 3, 1, 1).to(device)
    std = torch.tensor(std).view(1, 3, 1, 1).to(device)

    # Denormalize: x = x_norm * std + mean
    denorm_images = images * std + mean

    # Clamp to [0, 1] range to ensure valid pixel values
    return torch.clamp(denorm_images, 0.0, 1.0)

In [ ]:
import torch
import numpy as np

def batch_mse(pred, target):
    """Compute Mean Squared Error for a batch of images"""
    return torch.mean((pred - target)** 2, dim=[1, 2, 3])

scores = []
labels = []
types = []

with torch.no_grad():
    for batch in test_loader:

        batch_images = batch['image'].to(config.device)
        batch_images = denormalize(batch_images)

        batch_labels = batch['label']
        batch_types = batch['defect_type']

        reconstructed, _, _ = model(batch_images)
        batch_scores = batch_mse(reconstructed, batch_images)
        
        scores.extend(batch_scores.cpu().numpy())
        labels.extend(batch_labels.cpu().numpy())
        types.extend(batch_types)

scores = np.array(scores)
labels = np.array(labels)

print(scores.shape, labels.shape, len(types))

(83,) (83,) 83


In [ ]:
from sklearn.metrics import (roc_auc_score, 
    average_precision_score, f1_score, accuracy_score,
    precision_score, recall_score)

percentile = 95
threshold = float(np.percentile(scores, percentile))
predictions = (scores > threshold).astype(int)

auroc = float(roc_auc_score(labels, scores))
aupr = float(average_precision_score(labels, scores))
accuracy = float(accuracy_score(labels, predictions))
precision = float(precision_score(labels, predictions, zero_division=0))
recall = float(recall_score(labels, predictions, zero_division=0))
f1_score = float(f1_score(labels, predictions, zero_division=0))

auroc, aupr, accuracy, precision, recall, f1_score, threshold

(0.39841269841269844,
 0.7815537496157696,
 0.30120481927710846,
 1.0,
 0.07936507936507936,
 0.14705882352941177,
 0.0004987909342162311)

In [46]:
auroc, aupr, f1, acc, threshold

(0.39841269841269844,
 0.7815537496157696,
 0.14705882352941177,
 0.30120481927710846,
 0.0004987909342162311)